### Version Info
---

1. Numpy : 1.18.1
2. Pandas : 1.0.1

### Main Function
---
1. 입력
2. 변환<br>
    2-1 : 레이블 변환기 개발 필요: 가공치, 임플란트(Crown X), 임플란트(Crown O)인 경우에 상실치로 변경<br>
    2-2 : 레이블 변환기 개발 필요: 치관 탭 레이블 인덱스 변경<br>
    2-3 : 레이블 변환기 개발 필요: 기존 레이블 길이 변경 (악골병소 레이블을 1개에서 6개로 복사)<br>
    2-4 : 레이블 변환기 개발 필요: 헤더 추가<br>
    2-5 : 레이블 변환기 개발 필요: Disable된 레이블은 모두 -1로 변경<br>
3. 출력

## Basic Info
---
[Input]<br>
01 : Path<br>
02 : Filename<br>
03 ~ 34 : 치아 개수<br>
35 ~ 66 : 치관 정보<br>
67 ~ 98 : 충치 정보<br>
99 ~ 104 : 치주 질환<br>
105 : 약골 병소<br>
<br>
[Output]<br>
01 : Path<br>
02 : Filename<br>
03 ~ 34 : 치아 개수<br>
35 ~ 66 : 치관 정보<br>
67 ~ 98 : 충치 정보<br>
99 ~ 104 : 치주 질환<br>
105 ~ 110 : 약골 병소<br>



In [1]:
import numpy as np
import pandas as pd

In [448]:
# 01. Input
inputPath = r'E:\DataSet\DentalScreening\test\label.csv'

In [447]:
# 02. 변환
    # 02 - 1 : 파일 불러오기, 헤더 추가
################################################################################################
def loadNaddHeader(csvPath):
    def makeHeader():
        header = []
        header.append('path')
        header.append('Filename')
        # 치아개수 추가
        for i in range(1, 33):
            temp = "%02d_count"%i
            header.append(temp)
        # 치아상태 추가
        for i in range(1, 33):
            temp = "%02d_status"%i
            header.append(temp)
        # 충치상태 추가
        for i in range(1, 33):
            temp = "%02d_caries"%i    
            header.append(temp)
        # 치주질환 추가
        for i in range(1, 7):
            temp = "%02d_PD"%i    
            header.append(temp)
        # 약골병소 추가
        header.append('01_weakbone')
        return header
            
    header = makeHeader()
    # 일일이 지정해줄때
#    columns_name = ['Path','Filename',
#                    # 치아개수 01
#                    '01count','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1',
#                    # 치아개수 02
#                    '1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1',
#                    # 치관정보 01
#                    '1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1',
#                    # 치관정보 02
#                    '1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1',
#                    # 충치정보 01
#                    '1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1',
#                    # 충치정보 02
#                    '1','1','1','1','1','1','1','1','1','1','1','1','1','1','1','1',
#                    # 치주질환
#                    '1','1','1','1','2','3',
#                    # 약골병소
#                    '01_weakbone'
#                    ]
    csv = pd.read_csv(csvPath,names=header)
    csv_return = pd.DataFrame.copy(csv)
    
    return csv_return
################################################################################################

    # 02 - 2 : 가공치(8), 임플란트(crown O)(6), 임플란트(crown X)(7)일경우, 잔존치(1,2,3,4) -> 상실치(0) 변환
################################################################################################
def remainToMissing(csv):
    csv_return = pd.DataFrame.copy(csv)
    count_col = csv_return.columns[2:2+32]
    status_col = csv_return.columns[34:34+32]
    for idx in range(len(status_col)):
        csv_return[count_col[idx]] = np.where(csv_return[status_col[idx]]>5,0,csv_return[count_col[idx]])
    return csv_return   
################################################################################################ 

    # 02 - 3 : 치관 탭 레이블 인덱스 변경
    # (6->1), (7->2), (8->3), (2->4), (4->5), (5->6), (1->7), (3->8)
################################################################################################
def changeIdx(csv):
    csv_copy = pd.DataFrame.copy(csv)
    csv_return = pd.DataFrame.copy(csv)
    status_col = csv_copy.columns[34:34+32]
    for i in status_col:
        # csv_copy[i] <- 하나의 컬럼명 이 6이면 1로 아니면 0으로
        dst1 = np.where(csv_copy[i] == 6,1,0)
        dst2 = np.where(csv_copy[i] == 7,2,0)
        dst3 = np.where(csv_copy[i] == 8,3,0)
        dst4 = np.where(csv_copy[i] == 2,4,0)
        dst5 = np.where(csv_copy[i] == 4,5,0)
        dst6 = np.where(csv_copy[i] == 5,6,0)
        dst7 = np.where(csv_copy[i] == 1,7,0)
        dst8 = np.where(csv_copy[i] == 3,8,0)
        dst = dst1 + dst2 + dst3 + dst4 + dst5 + dst6 + dst7 + dst8
        csv_return[i] = dst
    return csv_return
################################################################################################

    # 02 - 4 : 기존 레이블 길이변경 (약골병소 레이블을 1개에서 6개로 복사 )
################################################################################################    
def weakbone1to6(csv):
    csv_return = pd.DataFrame.copy(csv)
    csv_return['02_weakbone'] = csv_return['01_weakbone']
    csv_return['03_weakbone'] = csv_return['01_weakbone']
    csv_return['04_weakbone'] = csv_return['01_weakbone']
    csv_return['05_weakbone'] = csv_return['01_weakbone']
    csv_return['06_weakbone'] = csv_return['01_weakbone']
    return csv_return
################################################################################################

    # 02 - 5 : 레이블 변환기 개발 필요: Disable된 레이블은 모두 -1로 변경
################################################################################################
def fill_1(csv):
    csv_return = pd.DataFrame.copy(csv)
    caries_col = csv_return.columns[66:]
    csv_return[caries_col] = -1
    return csv_return
################################################################################################

def runner(csvPath):
    csv1 = loadNaddHeader(csvPath)
    csv2 = weakbone1to6(csv1)
    csv3 = remainToMissing(csv2)
    csv4 = changeIdx(csv3)
    csv5 = fill_1(csv4)
    return csv5

In [449]:
# 02. 변환
result = runner(inputPath)

In [450]:
# 03. Output
outputPath = r'E:\DataSet\DentalScreening\test\label_changed.csv'
result.to_csv(outputPath)